In [1]:
#JV

In [2]:
from train import *
import wandb

In [3]:
"""WandB API Key Masked!!!"""

'WandB API Key Masked!!!'

In [4]:
# Creating a wandb sweep config

optim = Optimiser

sweep_config = {
    'method': 'grid',
    'name' : 'sweep cross entropy',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'number_of_hidden_layers': {
            'values': [3,4,5]
        },    
         'hidden_size':{
            'values':[32,64,128]
        },
        'activation': {
            'values': ['sigmoid','relu','tanh']
        },
        
        'initialization': {
            'values': ["random","xavier"]

        },
        'optimiser': {
            'values': ["sgd","gd_momentum","gd_nesterov","rmsprop","adam","nadam"]
        },
        
        'epochs': {
            'values': [5,10]
        },

        'batch_sizes': {
            'values': [16,32,64]
        },
        
        'lr': {
            'values': [1e-3,1e-4]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.5]
        },
        'loss': {
            'values': ['cross entropy']
        },


    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='JV_First_Run')

Create sweep with ID: pv2f75s4
Sweep URL: https://wandb.ai/tmajestical/JV_First_Run/sweeps/pv2f75s4


In [5]:
## train-test data is got from the mnist fashion import data call
## now this data is being flattened, i.e each image into a 1d array
## then flattened train data is split into 80% train and 20% validation data.

(x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()


seed = 76 #setting this as seed wherever randomness comes

x_train_flattened = x_train.flatten().reshape(x_train.shape[0],-1)/255
x_test_flattened = x_test.flatten().reshape(x_test.shape[0],-1)/255


#np.random.seed(seed)
random_num_generator = np.random.RandomState(seed)


validation_indices = random_num_generator.choice(x_train_flattened.shape[0],int(0.1*x_train_flattened.shape[0]),replace=False)
train_indices = np.array(list(set(np.arange(x_train_flattened.shape[0])).difference(set(validation_indices))))

x_train_data = x_train_flattened[train_indices]
y_train_data = y_train[train_indices]

x_validation_data = x_train_flattened[validation_indices]
y_validation_data = y_train[validation_indices]


In [6]:
def create_nnet_and_train(config):
    ##create a neural network.
    
    
    nn = NeuralNetwork(seed=seed)
    
    
    number_of_hidden_layers = config['number_of_hidden_layers']
    neurons_per_hidden_layer=[config['hidden_size']]*number_of_hidden_layers ## Assuming all layers have same number of neurons
    
    initialization = config['initialization']
    
    
    activation = config['activation']
    
    
    ## Create NNet with the current architecture config
    nn.createNNet(number_of_hidden_layers=number_of_hidden_layers,neurons_per_hidden_layer=neurons_per_hidden_layer,initialization = initialization,activation=activation)
    
    optim = Optimiser()
    
    
    loss_type=config['loss']
    optimiser_algo = config['optimiser']
    lr = config['lr']
    epochs = config['epochs']
    batch_size = config['batch_sizes']
    l2_param = config['weight_decay']
    
    ##train NNet.
    optim.train(nn,[x_train_data,y_train_data],[x_validation_data,y_validation_data],optimiser=optimiser_algo,lr=lr,epochs=epochs,batch_size=batch_size,l2_param=l2_param,print_val_accuracy=True,loss_type=loss_type,log_wandb_data=True)

In [ ]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''


    with wandb.init() as run:

        run_name="-hl_"+str(wandb.config.number_of_hidden_layers)+"-hs_"+str(wandb.config.hidden_size)+"-init_"+wandb.config.initialization+"-ac_"+wandb.config.activation

        run_name = run_name+"-optim_"+str(wandb.config.optimiser)+"-lr_"+str(wandb.config.lr) +"-epochs_"+str(wandb.config.epochs)+"-bs_"+str(wandb.config.batch_sizes)+"-reg_"+str(wandb.config.weight_decay)

        wandb.run.name=run_name

        create_nnet_and_train(wandb.config)
        

wandb.agent(sweep_id, function=main,count=200) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: zmn2t81p with config:
wandb: 	activation: sigmoid
wandb: 	batch_sizes: 16
wandb: 	epochs: 5
wandb: 	hidden_size: 32
wandb: 	initialization: random
wandb: 	loss: cross entropy
wandb: 	lr: 0.001
wandb: 	number_of_hidden_layers: 3
wandb: 	optimiser: sgd
wandb: 	weight_decay: 0
wandb: Currently logged in as: cs23m036 (tmajestical). Use `wandb login --relogin` to force relogin


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/54000 [00:00<?, ?it/s]

/Users/tejasmalladi/Documents/OM NAMO VENKATESAYA/Jai Vigneshwara IIT MADRAS/JV SEM2/JV Deep Learning/JV Assignments/JV-Deep-Learning-Assignment1/train.py:123: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Training   Accuracy : 24.13%  Avg. Loss : 1.99
Validation Accuracy : 23.43%  Avg. Loss : 1.99


...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/54000 [00:00<?, ?it/s]

/Users/tejasmalladi/Documents/OM NAMO VENKATESAYA/Jai Vigneshwara IIT MADRAS/JV SEM2/JV Deep Learning/JV Assignments/JV-Deep-Learning-Assignment1/train.py:123: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Training   Accuracy : 25.27%  Avg. Loss : 1.84
Validation Accuracy : 24.93%  Avg. Loss : 1.85


  0%|          | 0/54000 [00:00<?, ?it/s]